# 平均報酬強化学習

参考：
* [Model-free Reinforcement Learning in Infinite-horizon Average-reward Markov Decision Processes](https://arxiv.org/abs/1910.07072)

今回は平均報酬強化学習におけるモデルフリーな探索を見てみます．
基本的には，平均報酬強化学習を割引報酬強化学習で近似することでリグレットを導出していきます．

TODO: モデルベースなアルゴリズムは多分POLITEXあたりが良さそう？また今度．

表記：
* 平均総報酬：$J^\pi(s):=\liminf _{T \rightarrow \infty} \frac{1}{T} \mathbb{E}\left[\sum_{t=1}^T r\left(s_t, \pi\left(s_t\right)\right) \mid s_1=s\right]$
    * 最適ベルマン方程式：$J^*+q^*(s, a)=r(s, a)+\mathbb{E}_{s^{\prime} \sim p(\cdot \mid s, a)}\left[v^*\left(s^{\prime}\right)\right]$
* リグレット：$R_T:=\sum_{t=1}^T\left(J^*-r\left(s_t, a_t\right)\right)$
* Span：$\operatorname{sp}\left(v^*\right)=\max _s v^*(s)-\min _s v^*(s)$
* 割引報酬の表記：
$$
\begin{aligned}
\forall(s, a), & Q^*(s, a) & =r(s, a)+\gamma \mathbb{E}_{s^{\prime} \sim p(\cdot \mid s, a)}\left[V^*\left(s^{\prime}\right)\right] \\
\forall s, & V^*(s) & =\max _{a \in \mathcal{A}} Q^*(s, a) .
\end{aligned}
$$

---

仮定：

**Weakly communicating MDP**

MDPの状態が次の２つに分割できる場合，そのMDPのことをWeakly communicatingと呼びます．
* すべての状態が，任意の定常方策について，transientである．
* 任意の２つの状態を行き来できる定常方策が存在する．

この仮定がない場合は平均報酬でLow regretを達成するのは無理らしいです．

---

## Optimistic Q-learningアルゴリズム

次のアルゴリズムを考えます．

1. パラメータ：$H \geq 2$, $\delta \in (0, 1)$
2. 初期化：
    * $\gamma=1-\frac{1}{H}, \quad \forall s: \hat{V}_1(s)=H$
    * $\forall s, a: Q_1(s, a)=\hat{Q}_1(s, a)=H, \quad n_1(s, a)=0$
3. $\forall \tau, \alpha_\tau=\frac{H+1}{H+\tau}, b_\tau=4 \operatorname{sp}\left(v^*\right) \sqrt{\frac{H}{\tau} \ln \frac{2 T}{\delta}}$
4. For t = 1...T, do
    * $a_t=\operatorname{argmax}_{a \in \mathcal{A}} \hat{Q}_t\left(s_t, a\right)$
    * 次の更新をします：

$$
\begin{aligned}
& n_{t+1}\left(s_t, a_t\right) \leftarrow n_t\left(s_t, a_t\right)+1 \\
& \tau \leftarrow n_{t+1}\left(s_t, a_t\right) \\
& Q_{t+1}\left(s_t, a_t\right) \leftarrow\left(1-\alpha_\tau\right) Q_t\left(s_t, a_t\right) 
+\alpha_\tau\left[r\left(s_t, a_t\right)+\gamma \hat{V}_t\left(s_{t+1}\right)+b_\tau\right]\\
& \hat{Q}_{t+1}\left(s_t, a_t\right) \leftarrow \min \left\{\hat{Q}_t\left(s_t, a_t\right), Q_{t+1}\left(s_t, a_t\right)\right\} \\
& \hat{V}_{t+1}\left(s_t\right) \leftarrow \max _{a \in \mathcal{A}} \hat{Q}_{t+1}\left(s_t, a\right) .
\end{aligned}
$$

このアルゴリズムについて，次のリグレットが成立します：

---

**リグレットのバウンド**

Weakly communicatingなMDPについて，$H=\min \left\{\sqrt{\frac{\operatorname{sp}\left(v^*\right) T}{S A}},\left(\frac{T}{S A \ln \frac{4 T}{\delta}}\right)^{\frac{1}{3}}\right\}$のとき，上のアルゴリズムは次のリグレットを達成します：

$$
\mathcal{O}\left(\sqrt{\operatorname{sp}\left(v^*\right) S A T}+\operatorname{sp}\left(v^*\right)\left(T^{\frac{2}{3}}\left(S A \ln \frac{T}{\delta}\right)^{\frac{1}{3}}+\sqrt{T \ln \frac{1}{\delta}}\right)\right)
$$

---


**証明**

まず，リグレットを次のように分解します．

$$
\begin{aligned}
R_T= & \sum_{t=1}^T\left(J^*-r\left(s_t, a_t\right)\right) \\
= & \sum_{t=1}^T\left(J^*-(1-\gamma) V^*\left(s_t\right)\right) \\
& +\sum_{t=1}^T\left(V^*\left(s_t\right)-Q^*\left(s_t, a_t\right)\right) \\
& +\sum_{t=1}^T\left(Q^*\left(s_t, a_t\right)-\gamma V^*\left(s_t\right)-r\left(s_t, a_t\right)\right) .
\end{aligned}
$$

このそれぞれの項をバウンドしていきましょう．

---

**１つ目のバウンド**

まず，次が成立します（これちょっと怪しいかも．$\pi^*$の不等式の部分が怪しいぽい．でもLemma自体は大丈夫みたい．）：

1. $\left|J^*-(1-\gamma) V^*(s)\right| \leq(1-\gamma) \operatorname{sp}\left(v^*\right), \forall s \in \mathcal{S}$,
2. $\operatorname{sp}\left(V^*\right) \leq 2 \operatorname{sp}\left(v^*\right)$.

ベルマン方程式より，

$$
v^*(s)=r\left(s, \pi^*(s)\right)-J^*+\mathbb{E}_{s^{\prime} \sim p\left(\cdot \mid s, \pi^*(s)\right)} v^*\left(s^{\prime}\right)
$$

が成立します．ここで，$\pi^*$の割引設定におけるsub-optimalityを考えると，

$$
\begin{aligned}
V^*\left(s_1\right) & \geq \mathbb{E}\left[\sum_{t=1}^{\infty} \gamma^{t-1} r\left(s_t, \pi^*\left(s_t\right)\right) \mid s_1\right] \\
& =\mathbb{E}\left[\sum_{t=1}^{\infty} \gamma^{t-1}\left(J^*+v^*\left(s_t\right)-v^*\left(s_{t+1}\right)\right) \mid s_1\right] \\
& =\frac{J^*}{1-\gamma}+v^*\left(s_1\right)-\mathbb{E}\left[\sum_{t=2}^{\infty}\left(\gamma^{t-2}-\gamma^{t-1}\right) v^*\left(s_t\right) \mid s_1\right] \\
& \geq \frac{J^*}{1-\gamma}+\min _s v^*(s)-\max _s v^*(s) \sum_{t=2}^{\infty}\left(\gamma^{t-2}-\gamma^{t-1}\right) \\
& =\frac{J^*}{1-\gamma}-\operatorname{sp}\left(v^*\right),
\end{aligned}
$$

同様にして，

$$
\begin{aligned}
V^*\left(s_1\right) & =\mathbb{E}\left[\sum_{t=1}^{\infty} \gamma^{t-1} r\left(s_t, \pi_\gamma\left(s_t\right)\right) \mid s_1\right] \\
& \leq \mathbb{E}\left[\sum_{t=1}^{\infty} \gamma^{t-1}\left(J^*+v^*\left(s_t\right)-v^*\left(s_{t+1}\right)\right) \mid s_1\right] \\
& =\frac{J^*}{1-\gamma}+v^*\left(s_1\right)-\mathbb{E}\left[\sum_{t=2}^{\infty}\left(\gamma^{t-2}-\gamma^{t-1}\right) v^*\left(s_t\right) \mid s_1\right] \\
& \leq \frac{J^*}{1-\gamma}+\max _s v^*(s)-\min _s v^*(s) \sum_{t=2}^{\infty}\left(\gamma^{t-2}-\gamma^{t-1}\right) \\
& =\frac{J^*}{1-\gamma}+\operatorname{sp}\left(v^*\right),
\end{aligned}
$$

これで１つ目は証明完了です．続いて，

$$
\left|V^*\left(s_1\right)-V^*\left(s_2\right)\right| \leq\left|V^*\left(s_1\right)-\frac{J^*}{1-\gamma}\right|+\left|V^*\left(s_2\right)-\frac{J^*}{1-\gamma}\right| \leq 2 \operatorname{sp}\left(v^*\right)
$$

より，２つ目が成立します．


---

**３つ目のバウンド**

確率$1-\delta$以上で次が成立します．

$$
\begin{aligned}
& \sum_{t=1}^T\left(Q^*\left(s_t, a_t\right)-\gamma V^*\left(s_t\right)-r\left(s_t, a_t\right)\right) \\
& \leq 2 \operatorname{sp}\left(v^*\right) \sqrt{2 T \ln \frac{1}{\delta}}+2 \operatorname{sp}\left(v^*\right) .
\end{aligned}
$$

これはベルマン方程式とAzuma-Hoeffdingでいけます．


---

**２つ目のバウンド**

確率$1-\delta$以上で次が成立します．

$$
\begin{aligned}
& \sum_{t=1}^T\left(V^*\left(s_t\right)-Q^*\left(s_t, a_t\right)\right) \\
& \leq 4 H S A+24 \operatorname{sp}\left(v^*\right) \sqrt{H S A T \ln \frac{2 T}{\delta}}
\end{aligned}
$$


---
